<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline

import os
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Finite Volume Methods for Nonlinear Scalar Conservation Laws

As we saw previously, if we have a nonlinear conservation law that it be considered in this form or a form of nonuniqueness will result apart from the use of the weak form.  This extends to the numerical case as well and is critical to come to proper approximation of the weak solutions to the conservation law.

## Godunov's Method

From our original formulation of Godunov's method we obtained the numerical flux by evaluating the solution along the grid cell edge in time
$$
    F^n_{i-1/2} = f(Q_{i-1/2}).
$$
For nonlinear scalar problems there are 5 different configurations that may occur:
1. Shock traveling to the left
1. Rarefaction traveling fully to the left
1. A centered rarefaction
1. Rarefaction traveling fully to the right
1. Shock traveling to the right

In all but case 3 we know what value to pick to evaluate the flux function, i.e. $Q_{i-1}$ for case 1 and 2 and $Q_i$ for case 4 and 5.  For case 2 however it is no longer clear what value $Q$ to use.

Suppose that $f''(q) > 0$ everywhere such that if $Q_{i-1} < Q_i$ leads to a rarefaction.  If $Q_{i-1} < q_s < Q_i$ such that $q_s$ is the unique value where $f'(q_s) = 0$.  The value $q_s$ is often called a **stagnation point** as it propagates with speed 0.  It can also be called a **sonic point** as in gas dynamics (and shallow water) the speeds can by 0 only if the fluid speed $|u|$ and the sound speed (or gravity wave speed) equal.

With this being said we can generalize the Godunov flux function as
$$
    F^n_{i-1/2} = \left \{ \begin{aligned}
        &f(Q_{i-1}) & & \text{if } Q_{i-1} > q_s &\text{and } & s > 0 \\
        &f(Q_{i}) & & \text{if } Q_{i} < q_s &\text{and } & s < 0 \\
        &f(q_s) & & \text{if } Q_{i-1} < q_s < Q_i & &
    \end{aligned} \right .
$$
where $s$ is the Rankine-Hugoniot derived shock speed.

## Fluctuations, Waves, and Speeds

We still want to express Godunov's method in terms of our waves and fluctuations.  Recall our standard, first-order scheme
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (\mathcal{A}^+ \Delta Q_{i-1/2} + \mathcal{A}^- \Delta Q_{i+1/2} )
$$
where we define $\mathcal{A}^\pm \Delta Q_{i-1/2}$ as
$$\begin{aligned}
    &\mathcal{A}^+ \Delta Q_{i-1/2} = f(Q_i) - f(\widehat{Q}_{i-1/2}) \\
    &\mathcal{A}^- \Delta Q_{i-1/2} = f(\widehat{Q}_{i-1/2}) - f(Q_{i-1})
\end{aligned}$$

For the waves $\mathcal{W}_{i-1/2}$ and speeds $s_{i-1/2}$ that come into the higher-order correction terms we naturally have
$$
    \mathcal{W}_{i-1/2} = Q_i - Q_{i-1}
$$
and
$$
    s_{i-1/2} = \left \{ \begin{aligned}
        &\frac{f(Q_i) - f(Q_{i-1})}{Q_i - Q_{i-1}} & & \text{if } Q_{i-1} \neq Q_i \\
        &f'(Q_i) & & \text{if } Q_{i-1} = Q_i
    \end{aligned} \right .
$$

## Transonic Rarefactions and Entropy Fixes

Up until now we have only described a method that satisfies the Rankine-Hugoniot conditions but does not consider our previous case 3, a transonic/centered rarefaction.  If we were not to treat this case specially we would be violating our entropy conditions.

In order to handle this case we must apply what is mostly commonly termed an **entropy fix**.  An entropy fix aims to modify $\mathcal{A}^\pm \Delta Q_{i-1/2}$ such that it handles the transonic case and maintains an entropy satisfying solution.

In [ ]:
def true_solution(x, t):
    return numpy.zeros(x.shape)

def burgers_animation(order=2, efix=True):
   
    solver = pyclaw.ClawSolver1D(riemann.burgers_1D)

    solver.limiters = pyclaw.limiters.tvd.MC
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.order = order

    x = pyclaw.Dimension(-3.0, 3.0, 50, name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    state = pyclaw.State(domain, num_eqn)

    xc = domain.grid.x.centers
    state.q[0,:] = (xc < 0) * -numpy.ones(xc.shape) + 2.0 * (xc >= 0) * numpy.ones(xc.shape)
    state.problem_data['efix'] = efix

    claw = pyclaw.Controller()
    claw.tfinal = 6.0
    claw.num_output_times   = 30
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver

    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-1.5, 2.5))
    axes.set_title("Burgers Equation")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-1.5, 2.5))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        true_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'k-')
        return (computed_line, true_line)
    
    computed_line, true_line = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        true_line.set_data([x,], [true_solution(x, claw.frames[n].t)])
        return (computed_line, true_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)

In [ ]:
HTML(burgers_animation(order=2, efix=True).to_jshtml())

The practical application of an entropy fix usually consists of an indicator that the original solution may not be sufficient.  In other words in the case that there may exist a transonic rarefaction.  For scalar conservation laws this will occur when
$$
    f'(Q_{i-1}) < 0 < f'(Q_i)
$$
in which case we redefine $\mathcal{A}^\pm \Delta Q_{i-1/2}$ with
$$\begin{aligned}
    &\mathcal{A}^+ \Delta Q_{i-1/2} = f(Q_i) - f(q_s) \\
    &\mathcal{A}^- \Delta Q_{i-1/2} = f(q_s) - f(Q_i)
\end{aligned}$$

## Numerical Viscosity

It is evident that method mentioned above is not completely incorrect. How can this be the case when we know that the proper entropy satisfying solution should not be represented at all.  Instead we see a solution that partially does have the correct rarefaction structure.  What could be going on?

The fact that we are in reality getting a "pretty good" solution is that in reality the dominant error is providing a form of numerical viscosity that acts to create something like a physical viscosity that would match somewhat we want to approximate.  This suggests that, similar to finite-difference analysis, that what we really require is a form of regularization with some diffusion to obtain true solutions.

In the case of finite volume methods we can consider a numerical flux function
$$
    F_{i-1/2} = \frac{1}{2} [ f(Q_{i-1}) + f(Q_i) - |s_{i-1/2}| (Q_i - Q_{i-1})]
$$
that we know works as an unstable flux plus a diffusive like term.

## Lax-Friedrichs and Local Lax-Friedrichs Methods

A class of methods use a similar flux definition that was just mentioned previously.  The flux function is
$$
    F_{i-1/2} = \frac{1}{2} [f(Q_{i-1}) + f(Q_i) - a (Q_i - Q_{i-1})].
$$

As we have seen from finite difference and method of lines type analysis, this method has a numerical viscosity of
$$
    a = \frac{\Delta x}{\Delta t}
$$
with a fixed magnitude that at a sonic point will not vanish.  We can similarly define a local version of this that only applies the significant numerical viscosity at sonic points.  This can be defined as
$$
    F_{i-1/2} = \frac{1}{2} [f(Q_{i-1}) + f(Q_i) - a_{i-1/2} (Q_i - Q_{i-1})]
$$
where
$$
    a_{i-1/2} = \max(|f'(Q_{i-1}|, |f'(Q_i)|).
$$
This method is also called **Rusanov's Method**.

Another related method is **Murman's method** which uses
$$
    a_{i-1/2} = \left |\frac{f(Q_i) - f(Q_{i-1})}{Q_i - Q_{i-1}} \right|
$$

## Engquist-Osher method

We have constructed a method so far that always assumes a shock unless otherwise determined.  What if we chose the opposite and assume that everything is a shock instead?  This is known as the **Engquist-Osher method** and is accomplished by defining
$$\begin{aligned}
    \mathcal{A}^+ \Delta Q_{i-1/2} &= \int^{Q_i}_{Q_{i-1}} (f'(q))^+ dq \\
    \mathcal{A}^- \Delta Q_{i-1/2} &= \int^{Q_i}_{Q_{i-1}} (f'(q))^- dq
\end{aligned}$$

These are all implementable in our wave-propagation methodology where
$$\begin{aligned}
    \mathcal{A}^- \Delta Q_{i-1/2} &= \frac{1}{2} [f(Q_i) - f(Q_{i-1} - a_{i-1/2} (Q_i - Q_{i-1})] \\
    \mathcal{A}^+ \Delta Q_{i-1/2} &= \frac{1}{2} [f(Q_i) - f(Q_{i-1} + a_{i-1/2} (Q_i - Q_{i-1})]
\end{aligned}$$
that leads to flux functions of
$$\begin{aligned}
    F_{i-1/2} &= f(Q_{i-1}) + \int^{Q_i}_{Q_{i-1}} (f'(q))^- dq \\
    &= f(Q_i) - \int^{Q_i}_{Q_{i-1}} (f'(q))^+ dq \\
    &= \frac{1}{2} [f(Q_{i-1}) + f(Q_i)] - \frac{1}{2} \int^{Q_i}_{Q_{i-1}} |f'(q)| dq.
\end{aligned}$$

If $f'(q)$ does not change sign between $Q_{i-1}$ and $Q_i$ then one of these fluctuations will be zero and if it does, not only do we have a transonic rarefaction, but we also have the correct flux.

## E-Schemes

An **E-scheme** is one that satisfies
$$
    \text{sign}(Q_i - Q_{i-1}) [F_{i-1/2} - f(q)] \leq 0.
$$
One can show that Godunov's method is an E-scheme.

## High-Resolution TVD Methods

Extending our methods to higher-order is easy via the
$$
    Q_i^{n+1} = Q_i^n - \frac{\Delta t}{\Delta x} (\mathcal{A}^+\Delta Q_{i-1/2} + \mathcal{A}^-\Delta Q_{i+1/2}) - \frac{\Delta t}{\Delta x}(\widetilde{F}_{i+1/2} - \widetilde{F}_{i-1/2})
$$
where
$$
    \widetilde{F}_{i-1/2} = \frac{1}{2} |s_{i-1/2}| \left( 1 - \frac{\Delta t}{\Delta x} |s_{i-1/2}| \right ) \widetilde{\mathcal{W}}_{i-1/2}.
$$

## Conservation Form

Using the conservation form of an equation has some important properties.  If we have a solution that involves shocks we have already seen that manipulation of the non-conservative form can lead to different solutions.  This is also the basis of conservative forms of the numerical approximation also guaranteeing that we are finding true weak solutions to the original equation.

We can illustrate this idea by again considering Burgers' equation
$$
    u_t + \left(\frac{1}{2} u^2 \right)_x = 0.
$$
If $u > 0 \forall x \in \mathbb R$ then the conservative upwind method is
$$
    U^{n+1}_i = U^n_i - \frac{\Delta t}{\Delta x} \left( \frac{1}{2} (U^n_i)^2 - \frac{1}{2} (U^n_{i-1})^2 \right)
$$
where as the quasilinear form $u_t + uu_x = 0$ leads to
$$
    U^{n+1}_i = U^n_i - \frac{\Delta t}{\Delta x} U^n_i (U^n_i - U^n_{i-1}).
$$

For smooth solutions these are equivalent numerical methods.  Unfortunately for solutions that should contain a shock these two methods give divergent solutions.

We can actually write a direct discretization to the non-conservative form of the equation that looks much more complex:
$$
    U^{n+1}_i = U^n_i - \frac{\Delta t}{\Delta x} \left ( \frac{1}{2}(U^n_i)^2 - \frac{1}{2}(U^n_{i-1})^2 \right) + \frac{1}{2} \Delta t \Delta x \frac{\Delta t}{\Delta x} \left ( \frac{U^n_i - U^n_{i-1}}{\Delta x} \right)^2
$$

## Lax-Wendroff Theorem

One question that may come to mind is whether a consistent approximation to the conservation law will converge to a weak solution of said conservation law.  Lax and Wendroff proved as much.

Consider a sequence of grids indexed by $j=1, 2, \ldots$, with mesh parameters $\Delta t^{(j)}, \Delta x^{(j)} \rightarrow 0$ as $j \rightarrow \infty$.  Let $Q^{(j)}(x, t)$ denote the numerical approximation computed with a consistent and conservative methods on the $j$th grid.

Define convergence from a grid function $Q^{(j)}(x,t)$ to $q(x,t)$ in the following sense:
1. Over every bounded set $\Omega = [a,b] \times [0,T]$, then
$$
    \int^T_0 \int^a_b |Q^{(j)}(x,t) - q(x,t)| dx dt \rightarrow 0 \text{   as } j\rightarrow \infty
$$
This is the 1-norm over $\Omega$ and therefore can instead write this as
$$
    ||Q^{(j)} - q||_{1,\Omega} \rightarrow 0 \text{   as } j \rightarrow \infty.
$$
1. We also assume that for each $T \quad \exists R > 0$ s.t.
$$
    \text{TV}(Q^{(j)}) < R \quad \forall 0 \leq t \leq T, \quad j=1,2,\ldots
$$
where TV denotes the total variation function.

**Lax-Wendroff Theorem:**    Suppose that $Q^{(j)}$ converges to a function $q$ as $j \rightarrow \infty$, in the sense described above.  Then $q(x,t)$ is a weak solution of the conservation law.

To prove this theorem we will show that the limit function $q(x,t)$ satisfies the weak form for all $\phi \in C^1_0$.,
$$
    \int^\infty_0 \int^\infty_{-\infty} [\phi_t q + \phi_x f(q)] dx dt = - \int^\infty_{-\infty} \phi(x, 0) q(x, t) dx.
$$

Now define a grid version of $\phi$ so that we have $\Phi^{(j)}$ as an analogous gridded function.  Also define the relationship between subsequent grids by $\Phi^{(j)n}$ and similarly for $Q^{(j) n}_i$.  Now dropping the $j$ indexing but keeping the idea that $j \rightarrow \infty$ we move forward.

Take the original conservative method as
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{i+1/2} - F^n_{i-1/2} )
$$
and multiply by the gridded version of $\phi$ to find
$$
    \Phi^n_i Q^{n+1}_i = \Phi^n_i Q^n_i - \frac{\Delta t}{\Delta x} \Phi^n_i (F^n_{i+1/2} - F^n_{i-1/2} )
$$
that should be true for all values of $i$ and $n$ on each grid $j$.  Summing these over all $i$ and $n$ leads to
$$
    \sum^\infty_{n=0} \sum^\infty_{i=-\infty} \Phi^n_i(Q^{n+1}_i - Q^n_i) = -\frac{\Delta t}{\Delta x} \sum^\infty_{n=0} \sum^\infty_{i=-\infty} \Phi^n_i (F^n_{i+1/2} - F^n_{i-1/2}).
$$

Summation by parts (similar to integration by parts) leads to
$$
    - \sum^\infty_{-\infty} \Phi^0_i Q^0_i - \sum^\infty_{n=1} \sum^\infty_{i=-\infty} (\Phi^n_i - \Phi^{n-1}_i) Q^n_i = \frac{\Delta t}{\Delta x} \sum^\infty_{n=0} \sum^\infty_{i=-\infty} (\Phi^n_{i+1} - \Phi^n_i) F^n_{i-1/2}.
$$
Since $\Phi$ is compactly supported we in fact have the following:
$$
    \Delta x \Delta t \left [  \sum^\infty_{n=1} \sum^\infty_{i=-\infty} \left( \frac{\Phi^n_i - \Phi^{n-1}_{i}}{\Delta t} \right )Q^n_i + \sum^\infty_{n=0} \sum^\infty_{i=-\infty} \left( \frac{\Phi^n_{i+1} - \Phi^{n}_{i}}{\Delta x} \right )\right ] = -\Delta x \sum^\infty_{i=-\infty} \Phi^0_i Q^0_i
$$

At this point we can see that many of these terms go to zero as $j \rightarrow 0$.  The remaining flux function however are a bit more complex and lead to the additional assumptions that we have already imposed.  We then have
$$
    F^n_{i-1/2} \equiv F^{(j)n}_{i-1/2} = \mathcal{F}(Q^{(j)n}_{i-1}, Q^{(j)n}_i)
$$
with
$$
    |F^{(j)n}_{i-1/2} - f(Q^{(j)n}_i) | \leq L | Q^{(j)n}_{i} -Q^{(j)n}_{i-1},
$$
a clear Lipschitz condition.

If 
$$
    |F^{(j)n}_{i-1/2} - f(Q^{(j)n}_i) | \rightarrow 0 \text{   as } j\rightarrow \infty
$$
for almost all $i$ then we can say
$$
    \Delta x \Delta t \sum^\infty_{n=0} \sum^\infty_{i=-\infty} \left( \frac{\Phi^n_{i+1} - \Phi^{n}_{i}}{\Delta x} \right ) F^n_{i-1/2} \rightarrow \int^\infty_{0} \int^\infty_{-\infty} \phi_x(x,t) f(q(x,t)) dx dt
$$

## Entropy Conditions Revisited

### Entropy Consistency of Godunov Methods

## Nonlinear Stability